In [ ]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Abbas Muutasim
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset


100%|██████████| 211k/211k [00:00<00:00, 419MB/s]

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report


device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [13]:
# Step 1: Import pandas

# Step 2: Load the CSV file
# If using Colab, first upload the CSV file
# from google.colab import files
# uploaded = files.upload()  # choose spam.csv

df = pd.read_csv('/content/sms-spam-collection-dataset/spam.csv', encoding='latin-1')

# Step 3: Keep only relevant columns (label and text)
df = df[['v1', 'v2']]  # original dataset has extra unused columns
df.columns = ['label', 'text']  # rename for clarity
df['label'] = df['label'].replace({'ham': 'not spam', 'spam': 'spam'})


# Step 4: Display the first 10 rows
print("First 10 messages in the dataset:")
print(df.head(10))

First 10 messages in the dataset:
      label                                               text
0  not spam  Go until jurong point, crazy.. Available only ...
1  not spam                      Ok lar... Joking wif u oni...
2      spam  Free entry in 2 a wkly comp to win FA Cup fina...
3  not spam  U dun say so early hor... U c already then say...
4  not spam  Nah I don't think he goes to usf, he lives aro...
5      spam  FreeMsg Hey there darling it's been 3 week's n...
6  not spam  Even my brother is not like to speak with me. ...
7  not spam  As per your request 'Melle Melle (Oru Minnamin...
8      spam  WINNER!! As a valued network customer you have...
9      spam  Had your mobile 11 months or more? U R entitle...


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Split dataset into features (X) and labels (y)
X = df['text']    # the messages
y = df['label']   # spam or ham

# 2. Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Convert text into numbers using TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

print("Vectorization complete. Shape of training data:", X_train_vect.shape)


Vectorization complete. Shape of training data: (4457, 7735)


In [ ]:
# training the  using Naive Bayes

# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 1. Create the model
model = MultinomialNB()

# 2. Train the model
model.fit(X_train_vect, y_train)

# 3. Make predictions on the test set
y_pred = model.predict(X_test_vect)

# 4. Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9623318385650225

Confusion Matrix:
 [[965   0]
 [ 42 108]]

Classification Report:
               precision    recall  f1-score   support

         ham       0.96      1.00      0.98       965
        spam       1.00      0.72      0.84       150

    accuracy                           0.96      1115
   macro avg       0.98      0.86      0.91      1115
weighted avg       0.96      0.96      0.96      1115



In [14]:
# Interactive spam detector
while True:
    msg = input("Enter a message (or type 'exit' to quit): ")
    if msg.lower() == 'exit':
        print("Exiting...")
        break
    # Convert the message into numbers
    msg_vect = vectorizer.transform([msg])
    # Predict
    pred = model.predict(msg_vect)[0]
    print(f"Prediction: {pred}\n")


Enter a message (or type 'exit' to quit): congratulations you won 10000$ claim now
Prediction: spam

Enter a message (or type 'exit' to quit): are you comming today
Prediction: ham

Enter a message (or type 'exit' to quit): exit
Exiting...
